In [3]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import json 


In [16]:
driver = webdriver.Chrome()
driver.get("https://www.airbnb.fr/s/paris/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-05-01&monthly_length=3&monthly_end_date=2024-08-01&price_filter_input_type=0&channel=EXPLORE&date_picker_type=calendar&source=structured_search_input_header&search_type=filter_change&price_filter_num_nights=6&checkin=2024-05-06&checkout=2024-05-12&adults=2&children=2&query=Paris%2C%20France&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=62139ace-d3f9-42a5-876f-3a810c907664&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MCwiaXRlbXNfb2Zmc2V0IjowLCJ2ZXJzaW9uIjoxfQ%3D%3D")

In [5]:
def get_text(xpath):
    return driver.find_element(By.XPATH, xpath).text

In [6]:
def click(xpath):
    driver.find_element(By.XPATH, xpath).click()

In [7]:
def next_page():
    driver.find_element(By.XPATH, "//a[@aria-label='Suivant']").click()

In [8]:
def switch():
    window_handles = driver.window_handles

    driver.switch_to.window(window_handles[-1])

In [9]:
def close():
    driver.close()
    switch()

In [10]:
def get_price():
    return get_text('/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[1]')

In [11]:
def get_equipments():

    driver.find_elements(By.XPATH,"//*[contains(text(), 'Afficher les')]")[0].click()
    time.sleep(.3)
    equipments = driver.find_elements(By.XPATH, ("//div[@class='rten07p atm_lo_1tcgj5g atm_le_1tcgj5g atm_5j_t94yts dir dir-ltr']"))
    equipments = [i.text for i in equipments if 'Indisponible' not in i.text]
    return equipments


In [22]:
def get_offers():
    return driver.find_elements(By.XPATH, "//div[@class='lxq01kf atm_9s_1txwivl atm_am_kyuy1d atm_ar_d67k9l l1tup9az atm_1p4glcj_1bp4okc dir dir-ltr']")

In [13]:
def go_to(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

In [26]:
switch()

In [27]:
time.sleep(2)

liste = []
for j in range(15):
    offers = get_offers()
    for i in range(len(offers)):
        print(i)
        time.sleep(.5)
        go_to(offers[i])
        offers[i].click()
        time.sleep(.1)
        switch()
        try:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[1]'))).click()
        except : 
            close()
            continue
        liste.append([])
        liste[-1].append(get_price())
        liste[-1].append(get_equipments())
        close()
    if j!=14:
        next_page()
    time.sleep(2)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


In [80]:
description = driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div").text
[i for i in description.split('\n') if ("·" in i and len(i)>=5) ][0].split('·')

['4 voyageurs ', ' 1 chambre ', ' 2 lits ', ' 1 salle de bain']

In [66]:
dico = {}
dico['prix'] = []
for i in liste:
    for j in i[1]:
        dico[j] = []

In [67]:
for i in liste:
    prix = int(i[0].split()[0])
    equipements = i[1]
    dico['prix'].append(prix)
    for j in dico:
        if j != "prix":
            if j in equipements:
                dico[j].append(1)
            else:
                dico[j].append(0)
    

In [106]:
c=0
for i in list(dico):

    if "four" in i.lower():

        c+=1


c

37

In [114]:
words = {}

for i in list(dico):
    q = i.split()

    for j in q :
        if j.lower() not in words:
            words[j.lower()] = 1
        else:
            words[j.lower()] += 1

In [115]:
{k: v for k, v in sorted(words.items(), key=lambda item: item[1])}


{'prix': 1,
 'sèche-cheveux': 1,
 'produits': 1,
 'nettoyage': 1,
 'eau': 1,
 'chaude': 1,
 'cintres': 1,
 'fer': 1,
 'repasser': 1,
 'étendoir': 1,
 'quoi': 1,
 'lire': 1,
 'société': 1,
 'aire': 1,
 'extérieur': 1,
 'structures': 1,
 'central': 1,
 "d'appoint": 1,
 'commun': 1,
 'où': 1,
 'cuisiner': 1,
 'micro-ondes': 1,
 'casseroles': 1,
 'poêles,': 1,
 'huile,': 1,
 'sel': 1,
 'poivre': 1,
 'couverts': 1,
 'bols,': 1,
 'baguettes,': 1,
 'assiettes,': 1,
 'tasses,': 1,
 'bouilloire': 1,
 'verres': 1,
 'vin': 1,
 'cuisson': 1,
 'compacteur': 1,
 'déchets': 1,
 'manger': 1,
 'rendre': 1,
 'empruntant': 1,
 'sentier': 1,
 'quai': 1,
 'laverie': 1,
 'automatique': 1,
 'maison': 1,
 'dispose': 1,
 'd’un': 1,
 'profondeur': 1,
 '132cm': 1,
 'largeur': 1,
 '81cm.': 1,
 'dépôt': 1,
 'bagages': 1,
 'autorisé': 1,
 'confort': 1,
 'cas': 1,
 "d'arrivée": 1,
 'anticipée': 1,
 'départ': 1,
 'tardif': 1,
 'longue': 1,
 'durée': 1,
 'séjour': 1,
 'arrivée': 1,
 'autonome': 1,
 'boîte': 1,
 'clé':

In [68]:
df = pd.DataFrame(dico)

In [69]:
df

,prix,Baignoire,Sèche-cheveux,Produits de nettoyage,Eau chaude,Lave-linge (Payant) dans le logement,Cintres,Draps,Fer à repasser,Étendoir à linge,...,Four simple Brandt,Bidet,Sèche-linge (Gratuit),"TV HD avec Netflix, Disney+, Amazon Prime Video, télévision par câble haut de gamme",Réfrigérateur Sharp,Accès au complexe hôtelier\nLes voyageurs peuvent utiliser les installations du complexe à proximité,Abri voiture gratuit sur place,"Espace de travail dédié\nDans une partie du logement privée avec moniteur, chaise ergonomique et imprimante",Four lg,Livres et jouets pour enfants de Jusqu'à 2 ans et Entre 2 et 5 ans
0,90,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,95,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,141,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,132,0,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,89,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,120,0,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
267,106,0,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
268,180,0,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
269,150,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error as mse

y = df['prix']
X = df.drop('prix', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse(y_test, y_pred)**.5


68.67393312537946

In [98]:
y_test.mean()

128.4909090909091

In [99]:
mse(y_test, y_pred)**.5/y_test.mean()

0.5344653066217447

In [100]:
df.columns

Index(['prix', 'Baignoire', 'Sèche-cheveux', 'Produits de nettoyage',
       'Eau chaude', 'Lave-linge (Payant) dans le logement', 'Cintres',
       'Draps', 'Fer à repasser', 'Étendoir à linge',
       ...
       'Four simple Brandt', 'Bidet', 'Sèche-linge (Gratuit)',
       'TV HD avec Netflix, Disney+, Amazon Prime Video, télévision par câble haut de gamme',
       'Réfrigérateur Sharp',
       'Accès au complexe hôtelier\nLes voyageurs peuvent utiliser les installations du complexe à proximité',
       'Abri voiture gratuit sur place',
       'Espace de travail dédié\nDans une partie du logement privée avec moniteur, chaise ergonomique et imprimante',
       'Four lg',
       'Livres et jouets pour enfants de Jusqu'à 2 ans et Entre 2 et 5 ans'],
      dtype='object', length=426)